Загружаем данные

In [165]:
import numpy as np
from sklearn.datasets import load_iris
iris = load_iris()

In [166]:
X = iris.data[50:,:]
y = iris.target[50:]-1
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1])

In [167]:
X = np.c_[np.ones(len(X)), X]
X

array([[ 1. ,  7. ,  3.2,  4.7,  1.4],
       [ 1. ,  6.4,  3.2,  4.5,  1.5],
       [ 1. ,  6.9,  3.1,  4.9,  1.5],
       [ 1. ,  5.5,  2.3,  4. ,  1.3],
       [ 1. ,  6.5,  2.8,  4.6,  1.5],
       [ 1. ,  5.7,  2.8,  4.5,  1.3],
       [ 1. ,  6.3,  3.3,  4.7,  1.6],
       [ 1. ,  4.9,  2.4,  3.3,  1. ],
       [ 1. ,  6.6,  2.9,  4.6,  1.3],
       [ 1. ,  5.2,  2.7,  3.9,  1.4],
       [ 1. ,  5. ,  2. ,  3.5,  1. ],
       [ 1. ,  5.9,  3. ,  4.2,  1.5],
       [ 1. ,  6. ,  2.2,  4. ,  1. ],
       [ 1. ,  6.1,  2.9,  4.7,  1.4],
       [ 1. ,  5.6,  2.9,  3.6,  1.3],
       [ 1. ,  6.7,  3.1,  4.4,  1.4],
       [ 1. ,  5.6,  3. ,  4.5,  1.5],
       [ 1. ,  5.8,  2.7,  4.1,  1. ],
       [ 1. ,  6.2,  2.2,  4.5,  1.5],
       [ 1. ,  5.6,  2.5,  3.9,  1.1],
       [ 1. ,  5.9,  3.2,  4.8,  1.8],
       [ 1. ,  6.1,  2.8,  4. ,  1.3],
       [ 1. ,  6.3,  2.5,  4.9,  1.5],
       [ 1. ,  6.1,  2.8,  4.7,  1.2],
       [ 1. ,  6.4,  2.9,  4.3,  1.3],
       [ 1. ,  6.6,  3. ,

Стандартизируем признаки

In [168]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

Делим на обучающую и тестовую выборки

In [169]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=11)

Функция логистической регрессии

In [170]:
def sigma(X, w):
    f = X @ w
    return 1 / (1 + np.exp(-f))

Обучаем регрессию методом градиентного спуска

In [171]:
%%time
iter = 300
learn_rate = 0.01
eps = 0.0001
n = len(X_train)
w, prew_grad = np.zeros(len(X[0])), np.zeros(len(X[0]))
for _ in range(iter):
    grad = ((sigma(X_train, w) - y_train) @ X_train) / n
    w -= learn_rate * grad
    if np.linalg.norm(abs(grad - prew_grad)) < eps:
        break
    prew_grad = grad

Wall time: 10 ms


Функция метрики

In [172]:
def accur(X,y):
    classif = np.where(sigma(X,w) > 0.5, 1, 0)
    res = 1 - sum(np.abs(classif - y)) / len(y)
    return int(res*100000)/100000

Проверяем точность предсказания на обучающей и тестовой выборках

In [173]:
accur(X_train,y_train)

0.94285

In [174]:
accur(X_test,y_test)

0.9

Обучаем методом RMSProp

In [177]:
%%time
iter = 300
epsilon = 1e-8
eps = 0.0001
EG = 0.01
gamma = 0.9
nu = 2
n = len(X_train)
w, prew_grad = np.zeros(len(X[0])), np.zeros(len(X[0]))
for _ in range(iter):
    grad = ((sigma(X_train, w) - y_train) @ X_train) / n
    EG = gamma * EG + (1 - gamma) * grad**2
    learn_rate = nu / np.sqrt(EG + epsilon)
    w -= learn_rate * grad
    if np.linalg.norm(abs(grad - prew_grad)) < eps:
        break
    prew_grad = grad

Wall time: 0 ns


Считаем метрику для RMSProp

In [178]:
accur(X_train,y_train)

0.98571

In [179]:
accur(X_test,y_test)

0.93333

Обучаем методом NAdam

In [180]:
%%time
iter = 300
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-8
eps = 0.0001
nu = 500
n = len(X_train)
v = []
m, w, prew_grad = np.zeros(len(X[0])), np.zeros(len(X[0])), np.zeros(len(X[0]))
for i in range(iter):
    grad = np.array(((sigma(X_train, w) - y_train) @ X_train) / n)
    m = beta1 * m + (1 - beta1) * grad
    list(map(lambda x: x * beta2, v))
    v.append(np.linalg.norm(grad))
    w -= nu / np.sqrt(max(v) + epsilon) * m
    if np.linalg.norm(abs(grad - prew_grad)) < eps:
        break
    prew_grad = grad

Wall time: 0 ns


Считаем метрику для NAdam

In [182]:
accur(X_train,y_train)

0.95714

In [183]:
accur(X_test,y_test)

0.93333

In [185]:
import pandas as pd
df = pd.DataFrame([['Метод градиентного спуска',0.9, 10],
                  ['RMSProp',0.93333, 0],
                  ['NAdam',0.93333, 0]], 
columns=['Метод','Метрика', 'Время, мс'])
df

,Метод,Метрика,"Время, мс"
0,Метод градиентного спуска,0.90000,10
1,RMSProp,0.93333,0
2,NAdam,0.93333,0


**Вывод**

На рассмотренном наборе данных все 3 алгоритма показали очень хороший результат. Из-за малого размера датасета не удалось полноценно изучить разницу в метриках и времени исполнения. Тем не менее, понятно, что адаптивные градиентные методы гораздо быстрее и эффективнее классического градиентного спуска. NAdam вышел на минимум всего за 5 итераций. 